In [55]:
import pandas as pd
from scipy.stats import kurtosis
import matplotlib.pyplot
import numpy as np
import seaborn as sb

In [56]:
df = pd.read_excel('./Grupo 01.xlsx')
#TODO: medidas de tendencia central
#TODO: medidas de variacion
#TODO: cuantitativas cosntruir un grafico
#TODO: tabla de frecuencia para las cualitativas
#TODO: grafico para cualitativas
#TODO: medidas de tendencia central para precio segun el parqueadero
#TODO: medias de tendencia central para mt2 segun si esta terminaod o no
#TODO: box-plot para las ultimas dos tareas

In [57]:
var_precio = df['precio']

'''Medidas de tendencia central'''
precio_mean = var_precio.mean() #317.03
precio_median = var_precio.median() #245
precio_mode = var_precio.mode() #220

'''Medidas de dispersion'''
var_precio.describe()  # min = 25 ; max = 1700 ; 25% = 160 ; 50% = 245 ; 75% = 380 ; count=347
precio_std = var_precio.std(ddof = 0) #247.51, valor poblacional no muestral
precio_varianza = var_precio.var(ddof = 0) #61262.03, valor poblacional no muestral
precio_asimetria = var_precio.skew() #2.39
precio_curtosis = kurtosis(var_precio, fisher= False) #10.04
precio_range = (var_precio.max() - var_precio.min()) #1675

In [58]:
var_mt2 = df['mt2']

'''Medidas de tendencia central'''
mt2_mean = var_mt2.mean() #120.01
mt2_median = var_mt2.median() #96
mt2_mode = var_mt2.mode() #60

'''Medidas de dispersion'''
var_mt2.describe()  # min = 26 ; max = 500 ; 25% = 72 ; 50% = 96 ; 75% = 140 ; count = 347;
mt2_std = var_mt2.std(ddof = 0) #73.89, valor poblacional no muestral
mt2_varianza = var_mt2.var(ddof = 0) #5459.73 ,valor poblacional no muestral
mt2_simetria = var_mt2.skew() #1.88
mt2_curtosis = kurtosis(var_mt2, fisher= False) #6.97
mt2_rango = (var_mt2.max() - var_mt2.min()) #474

In [59]:
var_estrato = df['estrato']

'''Medidas de tendencia central'''
estrato_mean = var_estrato.mean() #4.64
estrato_median = var_estrato.median() #5
estrato_mode = var_estrato.mode() #6

'''Medidas de dispersion'''
var_estrato.describe()  # min = 2 ; max = 6; 25% = 4 ; 50% = 5 ; 75% = 6 ; count = 347;
estrato_std = var_estrato.std(ddof = 0) #1.19, valor poblacional no muestral. Muestral : ddof = 1
estrato_varianza = var_estrato.var(ddof = 0) #1.421 ,valor poblacional no muestral. Muestral : ddof = 1
estrato_asimetria = var_estrato.skew() #-0.247
estrato_curtosis = kurtosis(var_estrato, fisher= False) #1.674
estrato_rango = (var_estrato.max() - var_estrato.min()) #4

In [60]:
var_alcobas = df['alcobas']

'''Medidas de tendencia central'''
alcobas_mean = var_alcobas.mean() #2.89
alcobas_median = var_alcobas.median() #3
alcobas_mode = var_alcobas.mode() #3

'''Medidas de dispersion'''
var_alcobas.describe()  # min = 1 ; max = 5; 25% = 3 ; 50% = 3 ; 75% = 3 ; count = 347;
alcobas_std = var_alcobas.std(ddof = 0) #0.639, valor poblacional no muestral. Muestral : ddof = 1
alcobas_varianza = var_alcobas.var(ddof = 0) #0.4087 ,valor poblacional no muestral. Muestral : ddof = 1
alcobas_asimetria = var_alcobas.skew() #-0.099
alcobas_curtosis = kurtosis(var_alcobas, fisher= False) #4.55
alcobas_rango = (var_alcobas.max() - var_alcobas.min()) #4

In [61]:
var_banos = df['banos']

'''Medidas de tendencia central'''
banos_mean =   var_banos.mean()#2.29
banos_median = var_banos.median() #2
banos_mode = var_banos.mode() #2

'''Medidas de dispersion'''
var_banos.describe()  # min = 1 ; max = 5; 25% = 2 ; 50% = 2 ; 75% = 3 ; count = 347;
banos_std = var_banos.std(ddof = 0) #0.777, valor poblacional no muestral. Muestral : ddof = 1
banos_varianza = var_banos.var(ddof = 0) #0.605 ,valor poblacional no muestral. Muestral : ddof = 1
banos_asimetria = var_banos.skew() #0.762
banos_curtosis = kurtosis(var_banos, fisher= False) #3.82
banos_rango = (var_banos.max() - var_banos.min()) #4

### Start using aproximation's rules for two decimals

In [62]:
var_administracion = df['administracion']

'''Medidas de tendencia central'''
administracion_mean =   var_administracion.mean()#0.26
administracion_median = var_administracion.median() #0.19
administracion_mode = var_administracion.mode() #0

'''Medidas de dispersion'''
var_administracion.describe()  # min = 0 ; max = 2.3; 25% = 0.09 ; 50% = 0.19 ; 75% = 0.36 ; count = 347;
adminstracion_std = var_administracion.std(ddof = 0) #0.26, valor poblacional no muestral. Muestral : ddof = 1
administracion_varianza = var_administracion.var(ddof = 0) #0.41 ,valor poblacional no muestral. Muestral : ddof = 1
administracion_asimetria = var_administracion.skew() #2.2
administracion_curtosis = kurtosis(var_administracion, fisher= False) #13.5
administracion_rango = (var_administracion.max() - var_administracion.min()) #2.3

In [63]:
var_avaluo = df['avaluo']

'''Medidas de tendencia central'''
avaluo_mean = var_avaluo.mean() #185.32
avaluo_median = var_avaluo.median() #133.7
avaluo_mode = var_avaluo.mode() #28.1 ; 74 ; 95 ; 159

'''Medidas de dispersion'''
var_avaluo.describe()  # min = 0.15 ; max = 1518.14 ; 25% = 73 ; 50% = 133.7 ; 75% = 231.75 ; count = 347;
avaluo_std = var_avaluo.std(ddof = 0) #176.3, valor poblacional no muestral
avaluo_varianza = var_avaluo.var(ddof = 0) #31074.6 ,valor poblacional no muestral
avaluo_simetria = var_avaluo.skew() #2.8
avaluo_curtosis = kurtosis(var_avaluo, fisher= False) #16
avaluo_rango = (var_avaluo.max() - var_avaluo.min()) #1517.9

### Cualitative variable types

In [75]:
def get_acum (values,acum,storaged):
    """Funcion que llena un arreglo <storaged> con los valores acumulados
       de la variable que el pasemos como parametro en <values>
    """
    for i in values:
        acum = acum + i
        storaged.append(acum)

def get_frec_ab (values):
    frec_ab = values.value_counts()
    frec_df = pd.DataFrame(frec_ab)
    frec_df.columns = ['Frecuencia_absoluta']
    return frec_df

def get_frec_rel (dataframe,source):
    dataframe['Frecuencia_relativa'] = round(dataframe['Frecuencia_absoluta'] * 100 / len(source),3)


In [82]:
var_ubicacion = df['ubicacion']


'''Frecuencia absoluta'''

frec_ubicacion = get_frec_ab(var_ubicacion)

'''Frecuencia relativa'''

get_frec_rel(frec_ubicacion,var_ubicacion)

'''Frecuencia absoluta acumulada'''
valores_frec_ab = frec_ubicacion['Frecuencia_absoluta'].values
acumulado_ab = 0
frec_ab_acum = []
#
get_acum(valores_frec_ab,acumulado_ab,frec_ab_acum)

frec_ubicacion['Frec_absoluta_acumulada'] = frec_ab_acum

'''Frecuencia relativa acumulada'''
valores_frec_rel = frec_ubicacion['Frecuencia_relativa'].values
acumulado_rel = 0
frec_rel_acum = []

get_acum(valores_frec_rel,acumulado_rel,frec_rel_acum)

frec_ubicacion['Frec_relativa_acumulada'] = frec_rel_acum
frec_ubicacion

,Frecuencia_absoluta,Frecuencia_relativa,Frec_absoluta_acumulada,Frec_relativa_acumulada
poblado,134,38.617,134,38.617
aburra sur,84,24.207,218,62.824
laureles,36,10.375,254,73.199
occidente,35,10.086,289,83.285
belen guayabal,34,9.798,323,93.083
centro,19,5.476,342,98.559
norte,5,1.441,347,100.000


In [85]:
var_balcon = df['balcon']


'''Frecuencia absoluta'''

frec_balcon = get_frec_ab(var_balcon)

'''Frecuencia relativa'''

get_frec_rel(frec_balcon,var_balcon)

'''Frecuencia absoluta acumulada'''
valores_frec_ab = frec_balcon['Frecuencia_absoluta'].values
acumulado_ab = 0
frec_ab_acum = []
#
get_acum(valores_frec_ab,acumulado_ab,frec_ab_acum)

frec_balcon['Frec_absoluta_acumulada'] = frec_ab_acum

'''Frecuencia relativa acumulada'''
valores_frec_rel = frec_balcon['Frecuencia_relativa'].values
acumulado_rel = 0
frec_rel_acum = []

get_acum(valores_frec_rel,acumulado_rel,frec_rel_acum)
frec_balcon['Frec_relativa_acumulada'] = frec_rel_acum

,Frecuencia_absoluta,Frecuencia_relativa,Frec_absoluta_acumulada,Frec_relativa_acumulada
si,256,73.775,256,73.775
no,91,26.225,347,100.000
